---

## 1. Introduction — The Directionality Question

So far, all our connectivity measures have been **symmetric**:

- **Correlation**: $r(X, Y) = r(Y, X)$
- **PLV**: $PLV(X, Y) = PLV(Y, X)$
- **Mutual Information**: $I(X; Y) = I(Y; X)$

But real relationships are often **directional**!

### Examples of Directional Influence

- 🎓 Teacher speaks → Student listens
- 👥 Leader acts → Follower responds
- 🧠 Brain region A drives → Brain region B responds

### The Key Question

> Does X **influence** Y, or does Y **influence** X, or both?

### Enter Transfer Entropy

**Transfer Entropy (TE)** measures directed information flow:

> *"How much does knowing X's PAST help predict Y's FUTURE?"*

This is critical for hyperscanning: **who influences whom** during social interaction?

💡 **Key insight**: Transfer entropy answers: Does information FLOW from X to Y?

---

## 2. Setup

In [ ]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
from numpy.typing import NDArray
from typing import Tuple, Optional, Dict
from scipy import signal
from scipy.ndimage import gaussian_filter1d
import sys
sys.path.append("../../..")

from src.colors import COLORS
from src.plotting import configure_plots
from src.information import (
    compute_entropy_continuous,
    compute_mutual_information,
    compute_joint_entropy
)

configure_plots()

---

## 3. Intuition — Prediction and Causality

### A Thought Experiment: Weather Prediction

Imagine you want to predict **tomorrow's weather** in your city:

1. **Using only your city's past weather** → Some accuracy
2. **Adding the neighboring city's past weather** → Better accuracy?

If adding the neighbor's data improves prediction, then:
- Information **flows** from the neighbor to your city
- The neighbor's weather contains **predictive information** about yours

### Wiener-Granger Causality (1956, 1969)

> "X **Granger-causes** Y if X's past helps predict Y's future, **beyond** Y's past alone"

⚠️ **Important**: This is not true causality! It's **predictive influence**.

### Transfer Entropy = Information-Theoretic Granger Causality

| Granger Causality | Transfer Entropy |
|-------------------|------------------|
| Linear regression | Mutual Information |
| Assumes linearity | No linearity assumption |
| F-test for significance | Captures nonlinear influences |

In [ ]:
# Visualization 1: Conceptual diagram of prediction improvement

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: Predict Y from Y's past only
ax = axes[0]
ax.set_xlim(0, 10)
ax.set_ylim(0, 10)

# Y's past
ax.add_patch(plt.Rectangle((1, 3), 3, 4, color=COLORS["signal_2"], alpha=0.7))
ax.text(2.5, 5, "Y past", ha="center", va="center", fontsize=14, fontweight="bold", color="white")

# Y's future (with uncertainty)
ax.add_patch(plt.Rectangle((6, 3), 3, 4, color=COLORS["signal_2"], alpha=0.3))
ax.add_patch(plt.Rectangle((6.5, 3.5), 2, 3, color=COLORS["signal_2"], alpha=0.5))
ax.text(7.5, 5, "Y future\n(uncertain)", ha="center", va="center", fontsize=12)

# Arrow
ax.annotate("", xy=(6, 5), xytext=(4, 5),
            arrowprops=dict(arrowstyle="->", lw=3, color="black"))
ax.text(5, 5.8, "Predict", ha="center", fontsize=11)

ax.set_title("Prediction from Y's past only", fontsize=14, fontweight="bold")
ax.axis("off")

# Right: Predict Y from Y's past AND X's past
ax = axes[1]
ax.set_xlim(0, 10)
ax.set_ylim(0, 10)

# X's past
ax.add_patch(plt.Rectangle((1, 6), 3, 2.5, color=COLORS["signal_1"], alpha=0.7))
ax.text(2.5, 7.25, "X past", ha="center", va="center", fontsize=14, fontweight="bold", color="white")

# Y's past
ax.add_patch(plt.Rectangle((1, 2), 3, 2.5, color=COLORS["signal_2"], alpha=0.7))
ax.text(2.5, 3.25, "Y past", ha="center", va="center", fontsize=14, fontweight="bold", color="white")

# Y's future (less uncertainty!)
ax.add_patch(plt.Rectangle((6, 3), 3, 4, color=COLORS["signal_2"], alpha=0.3))
ax.add_patch(plt.Rectangle((6.8, 4), 1.4, 2, color=COLORS["signal_2"], alpha=0.8))
ax.text(7.5, 5, "Y future\n(less uncertain!)", ha="center", va="center", fontsize=12, fontweight="bold")

# Arrows
ax.annotate("", xy=(6, 5), xytext=(4, 3.25),
            arrowprops=dict(arrowstyle="->", lw=2, color=COLORS["signal_2"]))
ax.annotate("", xy=(6, 5), xytext=(4, 7.25),
            arrowprops=dict(arrowstyle="->", lw=3, color=COLORS["signal_1"]))

ax.text(5, 7, "Extra info!", ha="center", fontsize=11, color=COLORS["signal_1"], fontweight="bold")

ax.set_title("Prediction from BOTH pasts — TE measures this improvement!", fontsize=14, fontweight="bold")
ax.axis("off")

plt.tight_layout()
plt.show()

print("💡 Transfer Entropy = How much X's past REDUCES uncertainty about Y's future")
print("   (beyond what Y's own past already tells us)")

---

## 4. From Mutual Information to Transfer Entropy

### The Problem with MI

Mutual Information $I(X; Y)$ measures shared information between **current** values.

But it ignores **time**! We want to know about **prediction**.

### The Solution: Consider Past and Future

**Notation**:
- $X_{past}$ = history of X: $X(t-\tau), X(t-2\tau), ...$
- $Y_{past}$ = history of Y: $Y(t-\tau), Y(t-2\tau), ...$  
- $Y_{future}$ = what we want to predict: $Y(t)$

### Transfer Entropy Definition

$$TE_{X \to Y} = I(Y_{future}; X_{past} \mid Y_{past})$$

Read as: *"Information that X's past provides about Y's future, **given** Y's past"*

This is **conditional mutual information** — the information gained BEYOND what Y's past already tells us.

In [ ]:
# Visualization 2: Timeline diagram showing embedding for TE

fig, ax = plt.subplots(figsize=(14, 6))

# Time axis
ax.set_xlim(0, 14)
ax.set_ylim(0, 8)

# Draw X signal timeline
y_x = 6
ax.plot([1, 13], [y_x, y_x], 'k-', linewidth=2)
ax.text(0.5, y_x, "X", fontsize=16, fontweight="bold", va="center", color=COLORS["signal_1"])

# X past samples
for i, t in enumerate([3, 5, 7]):
    ax.plot(t, y_x, 'o', markersize=15, color=COLORS["signal_1"])
    ax.text(t, y_x + 0.5, f"t-{3-i}τ" if i < 2 else "t-τ", ha="center", fontsize=10)

# X past bracket
ax.annotate("", xy=(2.5, y_x - 0.3), xytext=(7.5, y_x - 0.3),
            arrowprops=dict(arrowstyle="<->", color=COLORS["signal_1"], lw=2))
ax.text(5, y_x - 0.8, "$X_{past}$", ha="center", fontsize=14, color=COLORS["signal_1"], fontweight="bold")

# Draw Y signal timeline
y_y = 3
ax.plot([1, 13], [y_y, y_y], 'k-', linewidth=2)
ax.text(0.5, y_y, "Y", fontsize=16, fontweight="bold", va="center", color=COLORS["signal_2"])

# Y past samples
for i, t in enumerate([3, 5, 7]):
    ax.plot(t, y_y, 'o', markersize=15, color=COLORS["signal_2"])

# Y past bracket  
ax.annotate("", xy=(2.5, y_y - 0.3), xytext=(7.5, y_y - 0.3),
            arrowprops=dict(arrowstyle="<->", color=COLORS["signal_2"], lw=2))
ax.text(5, y_y - 0.8, "$Y_{past}$", ha="center", fontsize=14, color=COLORS["signal_2"], fontweight="bold")

# Y future (target)
ax.plot(10, y_y, 's', markersize=20, color=COLORS["signal_3"], zorder=5)
ax.text(10, y_y + 0.6, "$Y_{future}$", ha="center", fontsize=14, color=COLORS["signal_3"], fontweight="bold")
ax.text(10, y_y - 0.6, "t", ha="center", fontsize=12)

# Arrows showing information flow
ax.annotate("", xy=(9.5, y_y + 0.2), xytext=(7.5, y_y),
            arrowprops=dict(arrowstyle="->", color=COLORS["signal_2"], lw=2, ls="--"))
ax.annotate("", xy=(9.5, y_y + 0.3), xytext=(7.5, y_x - 0.5),
            arrowprops=dict(arrowstyle="->", color=COLORS["signal_1"], lw=3))

# Labels
ax.text(8.5, 4.8, "TE = Extra\ninfo from X!", fontsize=12, color=COLORS["signal_1"], 
        fontweight="bold", ha="center")

# Time arrow
ax.annotate("", xy=(13, 1), xytext=(1, 1),
            arrowprops=dict(arrowstyle="->", color="gray", lw=2))
ax.text(7, 0.5, "Time →", ha="center", fontsize=12, color="gray")

ax.axis("off")
ax.set_title("Transfer Entropy: Information Flow from X's Past to Y's Future", 
             fontsize=14, fontweight="bold")

plt.tight_layout()
plt.show()

print("📊 TE measures how much X's past helps predict Y's future,")
print("   BEYOND what Y's own past already provides.")

---

## 5. The Transfer Entropy Formula

### Conditional Mutual Information

$$I(A; B \mid C) = H(A \mid C) - H(A \mid B, C)$$

*"Information A provides about B, given we already know C"*

### Transfer Entropy Formula

$$TE_{X \to Y} = H(Y_t \mid Y_{past}) - H(Y_t \mid Y_{past}, X_{past})$$

**Interpretation**:
- $H(Y_t \mid Y_{past})$ = uncertainty about Y's future given **only** Y's past
- $H(Y_t \mid Y_{past}, X_{past})$ = uncertainty given **both** pasts
- **TE** = reduction in uncertainty from adding X's information

### Key Properties

| Property | Meaning |
|----------|---------|
| $TE \geq 0$ | Adding information can't hurt prediction |
| $TE = 0$ | X provides no additional predictive power |
| $TE_{X \to Y} \neq TE_{Y \to X}$ | **Asymmetric!** (unlike MI) |
| Units: bits | (when using $\log_2$) |

In [ ]:
# Visualization 3: Venn diagram for conditional MI / Transfer Entropy

fig, ax = plt.subplots(figsize=(10, 8))

from matplotlib.patches import Circle

# Three circles: Y_future, Y_past, X_past
r = 2.0

# Positions (overlapping)
c_yfut = Circle((0, 0), r, fill=False, edgecolor=COLORS["signal_3"], linewidth=3, label="$Y_{future}$")
c_ypast = Circle((-1.5, -1.5), r, fill=False, edgecolor=COLORS["signal_2"], linewidth=3, label="$Y_{past}$")
c_xpast = Circle((1.5, -1.5), r, fill=False, edgecolor=COLORS["signal_1"], linewidth=3, label="$X_{past}$")

ax.add_patch(c_yfut)
ax.add_patch(c_ypast)
ax.add_patch(c_xpast)

# Labels
ax.text(0, 1.5, "$Y_{future}$", ha="center", fontsize=14, fontweight="bold", color=COLORS["signal_3"])
ax.text(-3, -2.5, "$Y_{past}$", ha="center", fontsize=14, fontweight="bold", color=COLORS["signal_2"])
ax.text(3, -2.5, "$X_{past}$", ha="center", fontsize=14, fontweight="bold", color=COLORS["signal_1"])

# Highlight TE region (Y_future ∩ X_past \ Y_past)
# This is simplified - just annotate the concept
ax.annotate("TE", xy=(0.7, -0.3), fontsize=20, fontweight="bold", color="red",
            ha="center", va="center")
ax.annotate("", xy=(0.7, -0.5), xytext=(1.2, -1.2),
            arrowprops=dict(arrowstyle="->", color="red", lw=2))

# Explanation text
ax.text(0, -4.5, 
        "$TE_{X→Y} = I(Y_{future}; X_{past} | Y_{past})$\n"
        "= Information X's past shares with Y's future,\n"
        "that Y's past doesn't already provide",
        ha="center", fontsize=12, style="italic",
        bbox=dict(boxstyle="round", facecolor="lightyellow", alpha=0.8))

ax.set_xlim(-5, 5)
ax.set_ylim(-6, 4)
ax.set_aspect("equal")
ax.axis("off")
ax.set_title("Transfer Entropy as Conditional Mutual Information", fontsize=14, fontweight="bold")

plt.tight_layout()
plt.show()

---

## 6. Implementing Transfer Entropy

### Steps to Compute TE

1. **Define embedding parameters**:
   - $k$ = history length for Y (how many past samples)
   - $l$ = history length for X
   - $\tau$ = time delay between samples

2. **Create state vectors**:
   - $Y_{past}$ = $[Y(t-\tau), Y(t-2\tau), ..., Y(t-k\tau)]$
   - $X_{past}$ = $[X(t-\tau), X(t-2\tau), ..., X(t-l\tau)]$
   - $Y_{future}$ = $Y(t)$

3. **Estimate entropies** via binning or KNN

4. **Compute TE** = $H(Y_t | Y_{past}) - H(Y_t | Y_{past}, X_{past})$

In [ ]:
def create_embedding_vectors(
    x: np.ndarray,
    y: np.ndarray,
    k: int = 1,
    l: int = 1,
    tau: int = 1
) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    Create embedded state vectors for Transfer Entropy computation.
    
    Parameters
    ----------
    x : np.ndarray
        Source signal.
    y : np.ndarray
        Target signal.
    k : int
        History length for target (Y). Default is 1.
    l : int
        History length for source (X). Default is 1.
    tau : int
        Embedding delay in samples. Default is 1.
    
    Returns
    -------
    y_future : np.ndarray
        Target values at time t, shape (n_valid,).
    y_past : np.ndarray
        Target history vectors, shape (n_valid, k).
    x_past : np.ndarray
        Source history vectors, shape (n_valid, l).
    """
    n = len(x)
    
    # Maximum lookback needed
    max_lag = max(k, l) * tau
    
    # Valid indices (where we have full history)
    n_valid = n - max_lag
    
    # Initialize arrays
    y_future = np.zeros(n_valid)
    y_past = np.zeros((n_valid, k))
    x_past = np.zeros((n_valid, l))
    
    for i in range(n_valid):
        idx = i + max_lag  # Current time index
        
        # Y future (target)
        y_future[i] = y[idx]
        
        # Y past (k samples)
        for j in range(k):
            y_past[i, j] = y[idx - (j + 1) * tau]
        
        # X past (l samples)
        for j in range(l):
            x_past[i, j] = x[idx - (j + 1) * tau]
    
    return y_future, y_past, x_past


# Quick test
np.random.seed(42)
x_test = np.random.randn(100)
y_test = np.random.randn(100)

y_fut, y_past, x_past = create_embedding_vectors(x_test, y_test, k=2, l=2, tau=1)
print(f"Input length: {len(x_test)}")
print(f"Output shapes: y_future={y_fut.shape}, y_past={y_past.shape}, x_past={x_past.shape}")

In [ ]:
def compute_transfer_entropy(
    x: np.ndarray,
    y: np.ndarray,
    k: int = 1,
    l: int = 1,
    tau: int = 1,
    n_bins: int = 8
) -> float:
    """
    Compute Transfer Entropy from X to Y: TE_{X→Y}.
    
    TE_{X→Y} = H(Y_t | Y_past) - H(Y_t | Y_past, X_past)
    
    Parameters
    ----------
    x : np.ndarray
        Source signal.
    y : np.ndarray
        Target signal.
    k : int
        History length for target (Y). Default is 1.
    l : int
        History length for source (X). Default is 1.
    tau : int
        Embedding delay in samples. Default is 1.
    n_bins : int
        Number of bins per dimension for histogram. Default is 8.
    
    Returns
    -------
    float
        Transfer entropy from X to Y in bits.
    """
    # Create embedding vectors
    y_future, y_past, x_past = create_embedding_vectors(x, y, k, l, tau)
    
    n_samples = len(y_future)
    
    # Discretize all variables
    def discretize(arr: np.ndarray) -> np.ndarray:
        """Discretize array into bins."""
        if arr.ndim == 1:
            arr = arr.reshape(-1, 1)
        
        result = np.zeros(arr.shape, dtype=int)
        for col in range(arr.shape[1]):
            # Use percentile-based binning for robustness
            percentiles = np.linspace(0, 100, n_bins + 1)
            bin_edges = np.percentile(arr[:, col], percentiles)
            result[:, col] = np.digitize(arr[:, col], bin_edges[1:-1])
        
        return result
    
    # Discretize
    y_fut_d = discretize(y_future).flatten()
    y_past_d = discretize(y_past)
    x_past_d = discretize(x_past)
    
    # Convert multi-dimensional indices to single index
    def to_single_index(arr: np.ndarray) -> np.ndarray:
        """Convert multi-column discrete array to single index."""
        if arr.ndim == 1:
            return arr
        result = np.zeros(len(arr), dtype=int)
        multiplier = 1
        for col in range(arr.shape[1] - 1, -1, -1):
            result += arr[:, col] * multiplier
            multiplier *= n_bins
        return result
    
    y_past_idx = to_single_index(y_past_d)
    x_past_idx = to_single_index(x_past_d)
    
    # Combined index for (y_past, x_past)
    yx_past_idx = y_past_idx * (n_bins ** l) + x_past_idx
    
    # Compute entropies using histogram counts
    def entropy_from_joint(idx1: np.ndarray, idx2: np.ndarray) -> float:
        """Compute H(idx1 | idx2) = H(idx1, idx2) - H(idx2)."""
        # Joint entropy H(idx1, idx2)
        joint = np.ravel_multi_index((idx1, idx2), (idx1.max() + 1, idx2.max() + 1))
        _, joint_counts = np.unique(joint, return_counts=True)
        p_joint = joint_counts / n_samples
        H_joint = -np.sum(p_joint * np.log2(p_joint + 1e-12))
        
        # Marginal entropy H(idx2)
        _, marginal_counts = np.unique(idx2, return_counts=True)
        p_marginal = marginal_counts / n_samples
        H_marginal = -np.sum(p_marginal * np.log2(p_marginal + 1e-12))
        
        return H_joint - H_marginal
    
    # H(Y_t | Y_past)
    H_y_given_ypast = entropy_from_joint(y_fut_d, y_past_idx)
    
    # H(Y_t | Y_past, X_past)
    H_y_given_yxpast = entropy_from_joint(y_fut_d, yx_past_idx)
    
    # Transfer Entropy
    te = H_y_given_ypast - H_y_given_yxpast
    
    # TE should be non-negative
    return max(0.0, te)


# Test with independent signals (TE should be ~0)
np.random.seed(42)
x_indep = np.random.randn(2000)
y_indep = np.random.randn(2000)

te_indep = compute_transfer_entropy(x_indep, y_indep, k=1, l=1, tau=1)
print(f"TE for independent signals: {te_indep:.4f} bits (should be ~0)")

---

## 7. Example — Coupled Signals

Let's test TE on signals where we **know** the ground truth:
- X drives Y with some delay
- Y does NOT drive X

We expect:
- $TE_{X \to Y} > 0$ (X influences Y)
- $TE_{Y \to X} \approx 0$ (Y doesn't influence X)

In [ ]:
def generate_coupled_signals(
    n_samples: int,
    coupling_x_to_y: float = 0.5,
    coupling_y_to_x: float = 0.0,
    delay_samples: int = 5,
    noise_level: float = 0.3,
    seed: Optional[int] = None
) -> Tuple[np.ndarray, np.ndarray]:
    """
    Generate coupled AR-like signals with specified directional coupling.
    
    Parameters
    ----------
    n_samples : int
        Number of samples to generate.
    coupling_x_to_y : float
        Coupling strength from X to Y (0 to 1). Default is 0.5.
    coupling_y_to_x : float
        Coupling strength from Y to X (0 to 1). Default is 0.0.
    delay_samples : int
        Delay in samples for the coupling. Default is 5.
    noise_level : float
        Standard deviation of noise. Default is 0.3.
    seed : int, optional
        Random seed for reproducibility.
    
    Returns
    -------
    x : np.ndarray
        Source signal (or first signal if bidirectional).
    y : np.ndarray
        Target signal (or second signal if bidirectional).
    """
    if seed is not None:
        np.random.seed(seed)
    
    # Initialize signals
    x = np.zeros(n_samples)
    y = np.zeros(n_samples)
    
    # Generate AR(1) base processes
    ar_coef = 0.7
    
    for t in range(1, n_samples):
        # Autoregressive component
        x[t] = ar_coef * x[t-1] + noise_level * np.random.randn()
        y[t] = ar_coef * y[t-1] + noise_level * np.random.randn()
        
        # Coupling X → Y
        if t >= delay_samples and coupling_x_to_y > 0:
            y[t] += coupling_x_to_y * x[t - delay_samples]
        
        # Coupling Y → X
        if t >= delay_samples and coupling_y_to_x > 0:
            x[t] += coupling_y_to_x * y[t - delay_samples]
    
    return x, y


# Generate unidirectionally coupled signals (X → Y)
x_coupled, y_coupled = generate_coupled_signals(
    n_samples=3000,
    coupling_x_to_y=0.5,
    coupling_y_to_x=0.0,
    delay_samples=5,
    seed=42
)

print("Generated coupled signals: X → Y (unidirectional)")

In [ ]:
# Visualization 4: Coupled signals and TE asymmetry

# Compute TE in both directions
te_x_to_y = compute_transfer_entropy(x_coupled, y_coupled, k=1, l=1, tau=5)
te_y_to_x = compute_transfer_entropy(y_coupled, x_coupled, k=1, l=1, tau=5)

fig, axes = plt.subplots(2, 2, figsize=(14, 8))

# Top left: X signal
ax = axes[0, 0]
ax.plot(x_coupled[:500], color=COLORS["signal_1"], linewidth=0.8)
ax.set_title("Signal X (driver)", fontsize=12, fontweight="bold")
ax.set_xlabel("Samples")
ax.set_ylabel("Amplitude")
ax.grid(True, alpha=0.3)

# Top right: Y signal  
ax = axes[0, 1]
ax.plot(y_coupled[:500], color=COLORS["signal_2"], linewidth=0.8)
ax.set_title("Signal Y (driven by X)", fontsize=12, fontweight="bold")
ax.set_xlabel("Samples")
ax.set_ylabel("Amplitude")
ax.grid(True, alpha=0.3)

# Bottom left: Scatter plot
ax = axes[1, 0]
ax.scatter(x_coupled[:-5], y_coupled[5:], alpha=0.3, s=5, color=COLORS["signal_3"])
ax.set_xlabel("X(t)", fontsize=12)
ax.set_ylabel("Y(t+5)", fontsize=12)
ax.set_title("X predicts future Y (5 samples later)", fontsize=12, fontweight="bold")
ax.grid(True, alpha=0.3)

# Bottom right: TE bar chart
ax = axes[1, 1]
bars = ax.bar(["TE: X → Y", "TE: Y → X"], [te_x_to_y, te_y_to_x], 
              color=[COLORS["signal_1"], COLORS["signal_2"]], width=0.5)
ax.set_ylabel("Transfer Entropy (bits)", fontsize=12)
ax.set_title("TE Correctly Detects Direction!", fontsize=12, fontweight="bold")

# Add value labels
for bar, val in zip(bars, [te_x_to_y, te_y_to_x]):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
            f"{val:.3f}", ha="center", fontsize=12, fontweight="bold")

ax.set_ylim(0, max(te_x_to_y, te_y_to_x) * 1.3)
ax.grid(True, alpha=0.3, axis="y")

plt.tight_layout()
plt.show()

print(f"\n📊 Results:")
print(f"   TE(X → Y) = {te_x_to_y:.4f} bits  ← X DOES influence Y")
print(f"   TE(Y → X) = {te_y_to_x:.4f} bits  ← Y does NOT influence X")
print(f"\n✅ TE correctly identifies the unidirectional coupling!")

In [ ]:
# Visualization 5: TE scales with coupling strength

coupling_strengths = np.linspace(0, 0.8, 9)
te_values = []

for coupling in coupling_strengths:
    x, y = generate_coupled_signals(
        n_samples=3000,
        coupling_x_to_y=coupling,
        coupling_y_to_x=0.0,
        delay_samples=5,
        seed=42
    )
    te = compute_transfer_entropy(x, y, k=1, l=1, tau=5)
    te_values.append(te)

fig, ax = plt.subplots(figsize=(10, 6))

ax.plot(coupling_strengths, te_values, "o-", color=COLORS["signal_1"], 
        linewidth=2.5, markersize=10)
ax.fill_between(coupling_strengths, te_values, alpha=0.3, color=COLORS["signal_1"])

ax.set_xlabel("Coupling Strength (X → Y)", fontsize=12)
ax.set_ylabel("Transfer Entropy (bits)", fontsize=12)
ax.set_title("TE Increases with Coupling Strength", fontsize=14, fontweight="bold")
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("💡 TE scales monotonically with the true coupling strength!")

---

## 8. Bidirectional Coupling and Net TE

Real systems often have **bidirectional coupling** — both signals influence each other.

TE can detect both directions:
- $TE_{X \to Y} > 0$ AND $TE_{Y \to X} > 0$

### Net Transfer Entropy

$$Net_{X \to Y} = TE_{X \to Y} - TE_{Y \to X}$$

| Net TE | Interpretation |
|--------|----------------|
| Positive | X dominates (more influence X→Y) |
| Negative | Y dominates (more influence Y→X) |
| ≈ Zero | Balanced bidirectional coupling |

In [ ]:
def compute_net_transfer_entropy(
    x: np.ndarray,
    y: np.ndarray,
    k: int = 1,
    l: int = 1,
    tau: int = 1,
    n_bins: int = 8
) -> Dict[str, float]:
    """
    Compute TE in both directions and net flow.
    
    Parameters
    ----------
    x : np.ndarray
        First signal.
    y : np.ndarray
        Second signal.
    k, l, tau, n_bins : int
        TE parameters.
    
    Returns
    -------
    dict
        Contains 'te_x_to_y', 'te_y_to_x', 'net_te', 'dominant_direction'.
    """
    te_x_to_y = compute_transfer_entropy(x, y, k, l, tau, n_bins)
    te_y_to_x = compute_transfer_entropy(y, x, k, l, tau, n_bins)
    net_te = te_x_to_y - te_y_to_x
    
    if net_te > 0.01:
        dominant = "X → Y"
    elif net_te < -0.01:
        dominant = "Y → X"
    else:
        dominant = "Balanced"
    
    return {
        "te_x_to_y": te_x_to_y,
        "te_y_to_x": te_y_to_x,
        "net_te": net_te,
        "dominant_direction": dominant
    }


# Generate bidirectionally coupled signals
x_bidir, y_bidir = generate_coupled_signals(
    n_samples=3000,
    coupling_x_to_y=0.5,  # X influences Y
    coupling_y_to_x=0.2,  # Y also influences X (but less)
    delay_samples=5,
    seed=42
)

result = compute_net_transfer_entropy(x_bidir, y_bidir, k=1, l=1, tau=5)

print("📊 Bidirectional Coupling Analysis:")
print(f"   TE(X → Y) = {result['te_x_to_y']:.4f} bits")
print(f"   TE(Y → X) = {result['te_y_to_x']:.4f} bits")
print(f"   Net TE    = {result['net_te']:.4f} bits")
print(f"   Dominant  = {result['dominant_direction']}")

In [ ]:
# Visualization 6: Net TE as function of coupling strength ratio

def generate_simple_coupled_signals(
    n_samples: int,
    coupling_x_to_y: float,
    coupling_y_to_x: float,
    delay: int = 5,
    seed: int = 42
) -> Tuple[np.ndarray, np.ndarray]:
    """
    Generate simple coupled signals without feedback loops.
    Uses independent noise sources with lagged coupling.
    """
    rng = np.random.RandomState(seed)
    
    # Independent noise sources
    noise_x = rng.randn(n_samples)
    noise_y = rng.randn(n_samples)
    
    # X = noise + AR component
    x = np.zeros(n_samples)
    for t in range(1, n_samples):
        x[t] = 0.7 * x[t-1] + 0.5 * noise_x[t]
    
    # Y = noise + AR component + coupling from X (lagged)
    y = np.zeros(n_samples)
    for t in range(1, n_samples):
        y[t] = 0.7 * y[t-1] + 0.5 * noise_y[t]
        if t >= delay:
            y[t] += coupling_x_to_y * x[t - delay]
    
    # Now add Y→X coupling (using a separate pass to avoid feedback loop)
    if coupling_y_to_x > 0:
        x_coupled = x.copy()
        for t in range(delay, n_samples):
            x_coupled[t] += coupling_y_to_x * y[t - delay]
        x = x_coupled
    
    return x, y


# Symmetric progression of coupling values
# Each condition has a clear difference from the previous one
coupling_values = [
    (0.6, 0.0),   # Only X→Y (strong)
    (0.6, 0.2),   # X→Y dominant
    (0.4, 0.4),   # Balanced
    (0.2, 0.6),   # Y→X dominant
    (0.0, 0.6),   # Only Y→X (strong)
]

labels = ["X→Y\nonly", "X→Y\ndominant", "Balanced", "Y→X\ndominant", "Y→X\nonly"]

net_te_values = []
te_xy_values = []
te_yx_values = []

for i, (c_xy, c_yx) in enumerate(coupling_values):
    x, y = generate_simple_coupled_signals(
        n_samples=5000,
        coupling_x_to_y=c_xy,
        coupling_y_to_x=c_yx,
        delay=5,
        seed=42 + i  # Different seed per condition
    )
    
    result = compute_net_transfer_entropy(x, y, k=1, l=1, tau=5, n_bins=10)
    te_xy_values.append(result["te_x_to_y"])
    te_yx_values.append(result["te_y_to_x"])
    net_te_values.append(result["net_te"])

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

x_pos = np.arange(len(coupling_values))

# Left: Both TEs as grouped bars
ax = axes[0]
width = 0.35
bars1 = ax.bar(x_pos - width/2, te_xy_values, width, color=COLORS["signal_1"], label="TE: X → Y")
bars2 = ax.bar(x_pos + width/2, te_yx_values, width, color=COLORS["signal_2"], label="TE: Y → X")
ax.set_xticks(x_pos)
ax.set_xticklabels(labels, fontsize=10)
ax.set_ylabel("Transfer Entropy (bits)", fontsize=12)
ax.set_title("TE in Both Directions", fontsize=12, fontweight="bold")
ax.legend()
ax.grid(True, alpha=0.3, axis="y")

# Right: Net TE
ax = axes[1]
colors = [COLORS["signal_1"] if v > 0 else COLORS["signal_2"] for v in net_te_values]
bars = ax.bar(x_pos, net_te_values, color=colors, alpha=0.7, edgecolor="black", linewidth=1.5)
ax.axhline(y=0, color=COLORS["grid"], linestyle="--", linewidth=2)
ax.set_xticks(x_pos)
ax.set_xticklabels(labels, fontsize=10)
ax.set_ylabel("Net TE (X→Y minus Y→X)", fontsize=12)
ax.set_title("Net TE Indicates Dominant Direction", fontsize=12, fontweight="bold")
ax.grid(True, alpha=0.3, axis="y")

# Add annotations
ax.annotate("X dominates", xy=(0.1, 0.85), xycoords="axes fraction", fontsize=10, 
            color=COLORS["signal_1"], fontweight="bold")
ax.annotate("Y dominates", xy=(0.7, 0.15), xycoords="axes fraction", fontsize=10,
            color=COLORS["signal_2"], fontweight="bold")

plt.tight_layout()
plt.show()

# Print the coupling values used
print("Coupling configurations:")
for label, (c_xy, c_yx) in zip(labels, coupling_values):
    print(f"  {label.replace(chr(10), ' ')}: X→Y = {c_xy}, Y→X = {c_yx}")
print()
print("💡 Net TE correctly identifies the dominant information flow direction!")
print(f"   • X→Y only:     Net TE = {net_te_values[0]:+.3f} (positive)")
print(f"   • Balanced:     Net TE = {net_te_values[2]:+.3f} (near zero)")
print(f"   • Y→X only:     Net TE = {net_te_values[-1]:+.3f} (negative)")

---

## 9. Parameter Selection

Computing TE requires choosing several **embedding parameters**:

| Parameter | Symbol | What it controls |
|-----------|--------|------------------|
| **Y history length** | $k$ | How many past values of Y to consider |
| **X history length** | $l$ | How many past values of X to consider |
| **Embedding delay** | $\tau$ | Time spacing between past values |
| **Number of bins** | `n_bins` | Discretization resolution |

### Guidelines for Parameter Selection

**Embedding delay (τ)**:
- Should match the expected influence timescale
- For neural signals: typically 10-100 ms
- Too short: captures autocorrelation, not coupling
- Too long: misses the causal relationship

**History lengths (k, l)**:
- Start simple: k = l = 1
- Increase if system has longer memory
- More history = higher dimensionality = need MORE data!

**Number of bins**:
- Same trade-off as for MI
- Too few: poor resolution
- Too many: sparse histograms, high bias

**Critical constraint**: Total dimensions $(k + l + 1) \times n\_bins$ should be $\ll n\_samples$

In [ ]:
# Visualization 7: Effect of embedding delay (τ) on TE

# Generate signals with known delay
true_delay = 8  # True coupling delay in samples

x_delayed, y_delayed = generate_coupled_signals(
    n_samples=5000,
    coupling_x_to_y=0.5,
    coupling_y_to_x=0.0,
    delay_samples=true_delay,
    seed=42
)

# Scan over different τ values
tau_values = np.arange(1, 20)
te_vs_tau = []

for tau in tau_values:
    te = compute_transfer_entropy(x_delayed, y_delayed, k=1, l=1, tau=tau, n_bins=8)
    te_vs_tau.append(te)

fig, ax = plt.subplots(figsize=(10, 5))

ax.plot(tau_values, te_vs_tau, "o-", color=COLORS["signal_1"], linewidth=2, markersize=8)
ax.axvline(x=true_delay, color=COLORS["signal_4"], linestyle="--", linewidth=2, 
           label=f"True delay = {true_delay}")
ax.set_xlabel("Embedding Delay τ (samples)", fontsize=12)
ax.set_ylabel("Transfer Entropy (bits)", fontsize=12)
ax.set_title("TE Peaks at the True Coupling Delay", fontsize=12, fontweight="bold")
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Find peak
peak_tau = tau_values[np.argmax(te_vs_tau)]
print(f"💡 TE peaks at τ = {peak_tau} samples (true delay = {true_delay})")
print("   Scanning τ can help identify the coupling timescale!")

In [ ]:
# Visualization 8: Effect of history length (k) on TE

k_values = [1, 2, 3, 4, 5]
te_vs_k = []
te_vs_k_indep = []  # For independent signals (should stay low)

# Independent signals for comparison
np.random.seed(123)
x_ind = np.random.randn(5000)
y_ind = np.random.randn(5000)

for k in k_values:
    # Coupled signals
    te_coupled = compute_transfer_entropy(x_delayed, y_delayed, k=k, l=1, tau=true_delay, n_bins=6)
    te_vs_k.append(te_coupled)
    
    # Independent signals
    te_ind = compute_transfer_entropy(x_ind, y_ind, k=k, l=1, tau=1, n_bins=6)
    te_vs_k_indep.append(te_ind)

fig, ax = plt.subplots(figsize=(10, 5))

ax.bar(np.array(k_values) - 0.15, te_vs_k, width=0.3, color=COLORS["signal_1"], 
       label="Coupled signals", alpha=0.8)
ax.bar(np.array(k_values) + 0.15, te_vs_k_indep, width=0.3, color=COLORS["signal_2"],
       label="Independent signals", alpha=0.8)

ax.set_xlabel("History Length k", fontsize=12)
ax.set_ylabel("Transfer Entropy (bits)", fontsize=12)
ax.set_title("Effect of History Length on TE", fontsize=12, fontweight="bold")
ax.set_xticks(k_values)
ax.legend()
ax.grid(True, alpha=0.3, axis="y")

plt.tight_layout()
plt.show()

print("💡 Observations:")
print("   • For coupled signals: TE is relatively stable (the relationship exists)")
print("   • For independent signals: TE increases with k (bias from dimensionality!)")
print("   • Higher k = more dimensions = more spurious patterns = higher bias")

---

## 10. Bias and Significance Testing

As we saw above, TE suffers from **positive bias**, especially with:
- Higher dimensions (larger k, l)
- Fewer bins (undersampled histograms)
- Limited data

### The Problem

Even for **completely independent** signals, TE will be positive!

This is because we're estimating high-dimensional joint distributions from finite data.

### The Solution: Surrogate Testing

1. **Generate surrogates**: Shuffle the source signal X (destroys X→Y relationship)
2. **Compute TE on surrogates**: This gives the "null" TE (bias only)
3. **Compare**: Observed TE vs null distribution
4. **Effective TE**: $TE_{eff} = TE_{observed} - \mathbb{E}[TE_{null}]$

In [ ]:
def compute_te_surrogate(
    x: NDArray[np.float64],
    y: NDArray[np.float64],
    k: int = 1,
    l: int = 1,
    tau: int = 1,
    n_bins: int = 8,
    seed: Optional[int] = None
) -> float:
    """
    Compute TE with shuffled source signal (null hypothesis).
    
    Parameters
    ----------
    x : NDArray[np.float64]
        Source signal (will be shuffled).
    y : NDArray[np.float64]
        Target signal (kept intact).
    k : int
        History length for target.
    l : int
        History length for source.
    tau : int
        Embedding delay.
    n_bins : int
        Number of bins for discretization.
    seed : int, optional
        Random seed for reproducibility.
    
    Returns
    -------
    float
        TE computed on shuffled source (null TE).
    """
    rng = np.random.RandomState(seed)
    x_shuffled = rng.permutation(x)
    return compute_transfer_entropy(x_shuffled, y, k, l, tau, n_bins)


def te_significance_test(
    x: NDArray[np.float64],
    y: NDArray[np.float64],
    k: int = 1,
    l: int = 1,
    tau: int = 1,
    n_bins: int = 8,
    n_surrogates: int = 200,
    seed: Optional[int] = None
) -> Dict[str, float]:
    """
    Test significance of transfer entropy using surrogate distribution.
    
    Parameters
    ----------
    x : NDArray[np.float64]
        Source signal.
    y : NDArray[np.float64]
        Target signal.
    k : int
        History length for target.
    l : int
        History length for source.
    tau : int
        Embedding delay.
    n_bins : int
        Number of bins.
    n_surrogates : int
        Number of surrogates for null distribution.
    seed : int, optional
        Random seed.
    
    Returns
    -------
    dict
        Contains: te_observed, te_effective, null_mean, null_std, p_value
    """
    # Observed TE
    te_observed = compute_transfer_entropy(x, y, k, l, tau, n_bins)
    
    # Generate null distribution
    rng = np.random.RandomState(seed)
    null_te = []
    for i in range(n_surrogates):
        te_null = compute_te_surrogate(x, y, k, l, tau, n_bins, seed=rng.randint(100000))
        null_te.append(te_null)
    
    null_te = np.array(null_te)
    null_mean = np.mean(null_te)
    null_std = np.std(null_te)
    
    # Effective TE (bias-corrected)
    te_effective = te_observed - null_mean
    
    # P-value (one-tailed: observed > null)
    p_value = np.mean(null_te >= te_observed)
    
    return {
        "te_observed": te_observed,
        "te_effective": te_effective,
        "null_mean": null_mean,
        "null_std": null_std,
        "null_distribution": null_te,
        "p_value": p_value
    }


print("✓ TE significance testing functions defined")

In [ ]:
# Visualization 9: Significance testing example

# Test on coupled signals
result_coupled = te_significance_test(
    x_delayed, y_delayed, 
    k=1, l=1, tau=true_delay, n_bins=8,
    n_surrogates=200, seed=42
)

# Test on independent signals
result_indep = te_significance_test(
    x_ind, y_ind,
    k=1, l=1, tau=1, n_bins=8,
    n_surrogates=200, seed=42
)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: Coupled signals
ax = axes[0]
ax.hist(result_coupled["null_distribution"], bins=30, color=COLORS["signal_2"], 
        alpha=0.7, edgecolor="white", label="Null distribution")
ax.axvline(result_coupled["te_observed"], color=COLORS["signal_1"], linewidth=3, 
           label=f"Observed TE = {result_coupled['te_observed']:.3f}")
ax.axvline(result_coupled["null_mean"], color=COLORS["grid"], linestyle="--", 
           linewidth=2, label=f"Null mean = {result_coupled['null_mean']:.3f}")
ax.set_xlabel("Transfer Entropy (bits)", fontsize=12)
ax.set_ylabel("Count", fontsize=12)
ax.set_title(f"Coupled Signals (p = {result_coupled['p_value']:.3f})", 
             fontsize=12, fontweight="bold")
ax.legend(fontsize=9)
ax.grid(True, alpha=0.3)

# Right: Independent signals
ax = axes[1]
ax.hist(result_indep["null_distribution"], bins=30, color=COLORS["signal_2"], 
        alpha=0.7, edgecolor="white", label="Null distribution")
ax.axvline(result_indep["te_observed"], color=COLORS["signal_1"], linewidth=3,
           label=f"Observed TE = {result_indep['te_observed']:.3f}")
ax.axvline(result_indep["null_mean"], color=COLORS["grid"], linestyle="--",
           linewidth=2, label=f"Null mean = {result_indep['null_mean']:.3f}")
ax.set_xlabel("Transfer Entropy (bits)", fontsize=12)
ax.set_ylabel("Count", fontsize=12)
ax.set_title(f"Independent Signals (p = {result_indep['p_value']:.3f})", 
             fontsize=12, fontweight="bold")
ax.legend(fontsize=9)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n💡 Results:")
print(f"   Coupled signals:     TE_eff = {result_coupled['te_effective']:.4f} bits, p = {result_coupled['p_value']:.4f} ✓ Significant!")
print(f"   Independent signals: TE_eff = {result_indep['te_effective']:.4f} bits, p = {result_indep['p_value']:.4f} ✗ Not significant")

---

## 11. TE for Neural Signals

When applying TE to EEG/neural data, consider:

### Preprocessing
- **Band-pass filter** to frequency of interest
- TE on raw broadband can be noisy
- Often compute TE on amplitude envelope or phase

### Typical Parameters for EEG
| Parameter | Typical Range | Rationale |
|-----------|---------------|-----------|
| τ | 10-50 ms | Expected neural delay |
| k, l | 1-3 | Short memory for oscillations |
| n_bins | 6-10 | Balance bias and resolution |

### Frequency-Specific TE
Different frequency bands may show different information flow patterns:
- **Theta** (4-8 Hz): Memory, navigation
- **Alpha** (8-13 Hz): Attention, inhibition
- **Beta** (13-30 Hz): Motor, coordination
- **Gamma** (30+ Hz): Perception, binding

In [ ]:
from scipy.signal import butter, filtfilt

def compute_te_bandlimited(
    x: NDArray[np.float64],
    y: NDArray[np.float64],
    fs: float,
    band: Tuple[float, float],
    k: int = 1,
    l: int = 1,
    tau_ms: float = 20.0,
    n_bins: int = 8
) -> float:
    """
    Compute TE on band-limited signals.
    
    Parameters
    ----------
    x : NDArray[np.float64]
        Source signal.
    y : NDArray[np.float64]
        Target signal.
    fs : float
        Sampling frequency in Hz.
    band : tuple
        Frequency band (low_freq, high_freq) in Hz.
    k : int
        History length for target.
    l : int
        History length for source.
    tau_ms : float
        Embedding delay in milliseconds.
    n_bins : int
        Number of bins.
    
    Returns
    -------
    float
        Transfer entropy on band-limited signals.
    """
    # Design bandpass filter
    low, high = band
    nyq = fs / 2
    b, a = butter(4, [low / nyq, high / nyq], btype="band")
    
    # Filter signals
    x_filt = filtfilt(b, a, x)
    y_filt = filtfilt(b, a, y)
    
    # Convert tau from ms to samples
    tau_samples = max(1, int(tau_ms * fs / 1000))
    
    return compute_transfer_entropy(x_filt, y_filt, k, l, tau_samples, n_bins)


# Visualization 10: Band-specific TE

# Generate coupled neural-like signals
fs = 250  # Hz
duration = 20  # seconds
n_samples = int(fs * duration)

np.random.seed(42)
t = np.arange(n_samples) / fs

# Create broadband "neural" signals with directional coupling in alpha band
x_neural = np.random.randn(n_samples)
y_neural = np.random.randn(n_samples)

# Add alpha oscillation to x
alpha_freq = 10  # Hz
x_neural += 2 * np.sin(2 * np.pi * alpha_freq * t)

# Couple x to y with delay (in alpha band)
delay_samples = int(0.02 * fs)  # 20 ms delay
for i in range(delay_samples, n_samples):
    y_neural[i] += 0.5 * x_neural[i - delay_samples]

# Define frequency bands
bands = {
    "Theta (4-8 Hz)": (4, 8),
    "Alpha (8-13 Hz)": (8, 13),
    "Beta (13-30 Hz)": (13, 30),
    "Gamma (30-50 Hz)": (30, 50)
}

band_colors = [COLORS["theta"], COLORS["alpha"], COLORS["beta"], COLORS["gamma"]]

te_per_band = {}
for band_name, band_range in bands.items():
    te = compute_te_bandlimited(x_neural, y_neural, fs, band_range, 
                                 k=1, l=1, tau_ms=20.0, n_bins=8)
    te_per_band[band_name] = te

fig, ax = plt.subplots(figsize=(10, 5))

band_names = list(te_per_band.keys())
te_values = list(te_per_band.values())

bars = ax.bar(band_names, te_values, color=band_colors, alpha=0.8, edgecolor="black")
ax.set_ylabel("Transfer Entropy (bits)", fontsize=12)
ax.set_title("Frequency-Specific TE (X → Y)", fontsize=12, fontweight="bold")
ax.grid(True, alpha=0.3, axis="y")

# Highlight alpha (where coupling exists)
bars[1].set_edgecolor(COLORS["signal_4"])
bars[1].set_linewidth(3)

plt.tight_layout()
plt.show()

print("💡 TE is highest in Alpha band — where the coupling exists!")
print("   This demonstrates frequency-specific information flow.")

---

## 12. TE Connectivity Matrix

For multi-channel analysis, we compute TE between all pairs of channels.

**Key difference from symmetric measures**: The TE matrix is **NOT symmetric**!

$$M[i, j] = TE_{i \to j} \neq TE_{j \to i} = M[j, i]$$

### Interpretation
- **Rows**: Information *senders* (sources)
- **Columns**: Information *receivers* (targets)
- High row sum → channel broadcasts to many
- High column sum → channel receives from many

### Net Flow Matrix
$$Net[i, j] = TE_{i \to j} - TE_{j \to i}$$

This is **antisymmetric**: $Net[i, j] = -Net[j, i]$

In [ ]:
def compute_te_matrix(
    data: NDArray[np.float64],
    k: int = 1,
    l: int = 1,
    tau: int = 1,
    n_bins: int = 8
) -> NDArray[np.float64]:
    """
    Compute directed TE matrix for multi-channel data.
    
    Parameters
    ----------
    data : NDArray[np.float64]
        Multi-channel data of shape (n_channels, n_samples).
    k : int
        History length for target.
    l : int
        History length for source.
    tau : int
        Embedding delay.
    n_bins : int
        Number of bins.
    
    Returns
    -------
    NDArray[np.float64]
        TE matrix of shape (n_channels, n_channels).
        Entry [i, j] = TE from channel i to channel j.
        Diagonal is NaN.
    """
    n_channels = data.shape[0]
    te_matrix = np.full((n_channels, n_channels), np.nan)
    
    for i in range(n_channels):
        for j in range(n_channels):
            if i != j:
                te_matrix[i, j] = compute_transfer_entropy(
                    data[i], data[j], k, l, tau, n_bins
                )
    
    return te_matrix


def compute_net_te_matrix(te_matrix: NDArray[np.float64]) -> NDArray[np.float64]:
    """
    Compute net TE matrix from directed TE matrix.
    
    Parameters
    ----------
    te_matrix : NDArray[np.float64]
        Directed TE matrix from compute_te_matrix.
    
    Returns
    -------
    NDArray[np.float64]
        Net TE matrix: Net[i,j] = TE[i,j] - TE[j,i].
        Antisymmetric matrix.
    """
    return te_matrix - te_matrix.T


# Visualization 11: TE matrix example

# Create 5 channels with known connectivity structure
n_channels = 5
n_samples = 3000
channel_names = ["Ch1", "Ch2", "Ch3", "Ch4", "Ch5"]

np.random.seed(42)
data = np.random.randn(n_channels, n_samples)

# Add autoregressive structure
for ch in range(n_channels):
    for t in range(1, n_samples):
        data[ch, t] += 0.7 * data[ch, t-1]

# Add directional connections:
# Ch1 → Ch2 (strong)
# Ch1 → Ch3 (moderate)  
# Ch4 → Ch5 (strong)
delay = 5
coupling = 0.6

for t in range(delay, n_samples):
    data[1, t] += coupling * data[0, t - delay]      # Ch1 → Ch2
    data[2, t] += 0.3 * data[0, t - delay]           # Ch1 → Ch3
    data[4, t] += coupling * data[3, t - delay]      # Ch4 → Ch5

# Compute TE matrix
te_mat = compute_te_matrix(data, k=1, l=1, tau=delay, n_bins=8)
net_te_mat = compute_net_te_matrix(te_mat)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: Directed TE matrix
ax = axes[0]
im = ax.imshow(te_mat, cmap="viridis", aspect="equal")
ax.set_xticks(range(n_channels))
ax.set_yticks(range(n_channels))
ax.set_xticklabels(channel_names)
ax.set_yticklabels(channel_names)
ax.set_xlabel("Target (receiver)", fontsize=12)
ax.set_ylabel("Source (sender)", fontsize=12)
ax.set_title("Directed TE Matrix", fontsize=12, fontweight="bold")
plt.colorbar(im, ax=ax, label="TE (bits)")

# Right: Net TE matrix
ax = axes[1]
vmax = np.nanmax(np.abs(net_te_mat))
im = ax.imshow(net_te_mat, cmap="RdBu_r", aspect="equal", vmin=-vmax, vmax=vmax)
ax.set_xticks(range(n_channels))
ax.set_yticks(range(n_channels))
ax.set_xticklabels(channel_names)
ax.set_yticklabels(channel_names)
ax.set_xlabel("Channel j", fontsize=12)
ax.set_ylabel("Channel i", fontsize=12)
ax.set_title("Net TE Matrix (antisymmetric)", fontsize=12, fontweight="bold")
plt.colorbar(im, ax=ax, label="Net TE (i→j)")

plt.tight_layout()
plt.show()

print("💡 The TE matrix correctly identifies:")
print("   • Strong Ch1 → Ch2 connection")
print("   • Moderate Ch1 → Ch3 connection")
print("   • Strong Ch4 → Ch5 connection")
print("   Note: The matrix is NOT symmetric!")

---

## 13. TE for Hyperscanning

This is **THE** key application for this workshop!

### The Central Question
> **Who leads the interaction?**

### Inter-Brain Transfer Entropy
- $TE_{P1 \to P2}$: Information flow from Participant 1 → Participant 2
- $TE_{P2 \to P1}$: Information flow from Participant 2 → Participant 1
- **Net TE**: Reveals the dominant direction

### Applications in Social Neuroscience
| Scenario | What TE reveals |
|----------|-----------------|
| Leader-follower dynamics | Who initiates actions |
| Teacher-student | Direction of knowledge transfer |
| Therapist-patient | Interpersonal influence |
| Parent-child | Regulatory influence direction |

### Advantages of TE for Hyperscanning
- ✓ **Directed** (unlike PLV, coherence, MI)
- ✓ **Captures nonlinear** influences
- ✓ **Time-resolved** analysis possible

In [ ]:
# Visualization 12: Hyperscanning TE simulation

def compute_te_hyperscanning(
    data_p1: NDArray[np.float64],
    data_p2: NDArray[np.float64],
    k: int = 1,
    l: int = 1,
    tau: int = 1,
    n_bins: int = 8
) -> Dict[str, NDArray[np.float64]]:
    """
    Compute inter-brain TE for hyperscanning.
    
    Parameters
    ----------
    data_p1 : NDArray[np.float64]
        Participant 1 data, shape (n_channels, n_samples).
    data_p2 : NDArray[np.float64]
        Participant 2 data, shape (n_channels, n_samples).
    k, l : int
        History lengths.
    tau : int
        Embedding delay.
    n_bins : int
        Number of bins.
    
    Returns
    -------
    dict
        Contains: te_p1_to_p2, te_p2_to_p1, net_te matrices.
    """
    n_ch1 = data_p1.shape[0]
    n_ch2 = data_p2.shape[0]
    
    te_p1_to_p2 = np.zeros((n_ch1, n_ch2))
    te_p2_to_p1 = np.zeros((n_ch2, n_ch1))
    
    for i in range(n_ch1):
        for j in range(n_ch2):
            te_p1_to_p2[i, j] = compute_transfer_entropy(
                data_p1[i], data_p2[j], k, l, tau, n_bins
            )
            te_p2_to_p1[j, i] = compute_transfer_entropy(
                data_p2[j], data_p1[i], k, l, tau, n_bins
            )
    
    # Net TE: average across all pairs
    mean_p1_to_p2 = np.mean(te_p1_to_p2)
    mean_p2_to_p1 = np.mean(te_p2_to_p1)
    net_te = mean_p1_to_p2 - mean_p2_to_p1
    
    return {
        "te_p1_to_p2": te_p1_to_p2,
        "te_p2_to_p1": te_p2_to_p1,
        "mean_p1_to_p2": mean_p1_to_p2,
        "mean_p2_to_p1": mean_p2_to_p1,
        "net_te": net_te,
        "leader": "P1" if net_te > 0 else "P2"
    }


# Simulate hyperscanning scenario: P1 leads, P2 follows
n_channels = 3
n_samples = 4000
channel_names = ["Fz", "Cz", "Pz"]

np.random.seed(42)

# P1 generates independent signals
data_p1 = np.random.randn(n_channels, n_samples)
for ch in range(n_channels):
    for t in range(1, n_samples):
        data_p1[ch, t] += 0.7 * data_p1[ch, t-1]

# P2 is influenced by P1 with some delay
delay = 8
coupling = 0.5

data_p2 = np.random.randn(n_channels, n_samples)
for ch in range(n_channels):
    for t in range(1, n_samples):
        data_p2[ch, t] += 0.7 * data_p2[ch, t-1]
    # Add influence from corresponding P1 channel
    for t in range(delay, n_samples):
        data_p2[ch, t] += coupling * data_p1[ch, t - delay]

# Compute inter-brain TE
hyper_result = compute_te_hyperscanning(data_p1, data_p2, k=1, l=1, tau=delay, n_bins=8)

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Left: P1 → P2
ax = axes[0]
im = ax.imshow(hyper_result["te_p1_to_p2"], cmap="Blues", aspect="equal")
ax.set_xticks(range(n_channels))
ax.set_yticks(range(n_channels))
ax.set_xticklabels([f"P2-{ch}" for ch in channel_names])
ax.set_yticklabels([f"P1-{ch}" for ch in channel_names])
ax.set_title("TE: P1 → P2", fontsize=12, fontweight="bold", color=COLORS["signal_1"])
plt.colorbar(im, ax=ax)

# Middle: P2 → P1
ax = axes[1]
im = ax.imshow(hyper_result["te_p2_to_p1"], cmap="Reds", aspect="equal")
ax.set_xticks(range(n_channels))
ax.set_yticks(range(n_channels))
ax.set_xticklabels([f"P1-{ch}" for ch in channel_names])
ax.set_yticklabels([f"P2-{ch}" for ch in channel_names])
ax.set_title("TE: P2 → P1", fontsize=12, fontweight="bold", color=COLORS["signal_2"])
plt.colorbar(im, ax=ax)

# Right: Summary bar chart
ax = axes[2]
bars = ax.bar(["P1 → P2", "P2 → P1"], 
              [hyper_result["mean_p1_to_p2"], hyper_result["mean_p2_to_p1"]],
              color=[COLORS["signal_1"], COLORS["signal_2"]], alpha=0.8)
ax.set_ylabel("Mean TE (bits)", fontsize=12)
ax.set_title(f"Net TE = {hyper_result['net_te']:.3f}\nLeader: {hyper_result['leader']}", 
             fontsize=12, fontweight="bold")
ax.grid(True, alpha=0.3, axis="y")

plt.tight_layout()
plt.show()

print(f"💡 Inter-brain TE analysis:")
print(f"   P1 → P2: {hyper_result['mean_p1_to_p2']:.4f} bits (mean)")
print(f"   P2 → P1: {hyper_result['mean_p2_to_p1']:.4f} bits (mean)")
print(f"   Net TE: {hyper_result['net_te']:.4f} → {hyper_result['leader']} leads the interaction!")

---

## 14. Limitations and Cautions

### ⚠️ Critical Limitations

| Limitation | Description | Mitigation |
|------------|-------------|------------|
| **Data hungry** | TE needs MORE data than MI | Use shorter histories (k, l = 1) |
| **Parameter sensitive** | Results depend on k, l, τ, n_bins | Report sensitivity analysis |
| **NOT causality** | TE measures *prediction*, not causation | Cannot replace experiments |
| **Stationarity** | Assumes stable statistics | Use time-resolved TE |
| **Confounds** | Common driver creates apparent TE | Control for shared inputs |

### The Confound Problem

If a hidden variable **Z** drives both **X** and **Y**:

```
    Z
   / \
  ↓   ↓
  X   Y
```

Then TE will detect X→Y even if X doesn't actually cause Y!

> **TE is NOT causation** — it's predictive information flow.

---

## 15. Hands-On Exercises

### Exercise 1: Delay Detection
Create coupled signals with a known delay (e.g., 15 samples). Use TE scanning over τ to recover the true delay.

```python
# Your code here
true_delay = 15
# Generate signals...
# Scan τ from 1 to 25...
# Find the peak...
```

### Exercise 2: Bidirectional Coupling Analysis
Generate signals with asymmetric bidirectional coupling:
- X → Y with strength 0.6
- Y → X with strength 0.2

Compute Net TE and verify it correctly identifies X as the dominant driver.

### Exercise 3: Significance Testing
1. Generate independent signals
2. Compute TE (will be biased positive)
3. Run surrogate test — is it significant?
4. Compare with coupled signals

In [ ]:
# Exercise Solutions

# Exercise 1: Delay Detection
print("=" * 50)
print("Exercise 1: Delay Detection")
print("=" * 50)

true_delay_ex = 15
x_ex, y_ex = generate_coupled_signals(
    n_samples=4000,
    coupling_x_to_y=0.5,
    delay_samples=true_delay_ex,
    seed=123
)

tau_range_ex = np.arange(1, 26)
te_scan = [compute_transfer_entropy(x_ex, y_ex, k=1, l=1, tau=t, n_bins=8) 
           for t in tau_range_ex]

detected_delay = tau_range_ex[np.argmax(te_scan)]
print(f"True delay: {true_delay_ex} samples")
print(f"Detected delay (max TE): {detected_delay} samples")
print(f"Match: {'✓' if detected_delay == true_delay_ex else '✗'}")

# Exercise 2: Asymmetric Bidirectional Coupling
print("\n" + "=" * 50)
print("Exercise 2: Asymmetric Bidirectional Coupling")
print("=" * 50)

x_bidir_ex, y_bidir_ex = generate_simple_coupled_signals(
    n_samples=5000,
    coupling_x_to_y=0.6,
    coupling_y_to_x=0.2,
    delay=5,
    seed=456
)

result_bidir = compute_net_transfer_entropy(x_bidir_ex, y_bidir_ex, k=1, l=1, tau=5, n_bins=8)
print(f"TE X→Y: {result_bidir['te_x_to_y']:.4f} bits")
print(f"TE Y→X: {result_bidir['te_y_to_x']:.4f} bits")
print(f"Net TE: {result_bidir['net_te']:.4f}")
print(f"Dominant: {'X (correct!)' if result_bidir['net_te'] > 0 else 'Y'}")

# Exercise 3: Significance Testing
print("\n" + "=" * 50)
print("Exercise 3: Significance Testing")
print("=" * 50)

np.random.seed(789)
x_indep_ex = np.random.randn(3000)
y_indep_ex = np.random.randn(3000)

result_sig = te_significance_test(x_indep_ex, y_indep_ex, k=1, l=1, tau=1, 
                                   n_bins=8, n_surrogates=100, seed=42)
print(f"Independent signals:")
print(f"  Observed TE: {result_sig['te_observed']:.4f}")
print(f"  Null mean:   {result_sig['null_mean']:.4f}")
print(f"  Effective TE: {result_sig['te_effective']:.4f}")
print(f"  P-value: {result_sig['p_value']:.4f}")
print(f"  Significant at α=0.05: {'No ✓' if result_sig['p_value'] > 0.05 else 'Yes ✗'}")

---

## 16. Summary

### Key Takeaways

| Concept | Description |
|---------|-------------|
| **Transfer Entropy** | Measures *directed* information flow |
| **Formula** | $TE_{X \to Y} = I(Y_t; X_{past} \| Y_{past})$ |
| **Asymmetric** | $TE_{X \to Y} \neq TE_{Y \to X}$ |
| **Granger Causality** | Information-theoretic version (nonlinear) |
| **Embedding** | Requires k, l (history) and τ (delay) |
| **Bias** | Always use surrogate testing! |
| **Net TE** | $TE_{X \to Y} - TE_{Y \to X}$ indicates dominant direction |

### For Hyperscanning

Transfer Entropy is particularly valuable because it reveals **who leads the interaction**:
- Compute TE in both directions
- Net TE > 0 → Participant 1 leads
- Net TE < 0 → Participant 2 leads
- Time-resolved TE → Track leadership dynamics

### Critical Reminder

> **TE is NOT causation** — it measures predictive information, not true causal influence. Confounds can create spurious TE!

---

## Discussion Questions

1. You find $TE_{P1 \to P2} = 0.15$ bits and $TE_{P2 \to P1} = 0.08$ bits during a cooperative task. What does this tell you about the interaction?

2. TE at τ = 50ms is higher than at τ = 100ms. What might this indicate about neural communication?

3. How would you respond to: "TE is just fancy correlation — you're not proving causation"?

4. Your analysis shows P1→P2 dominance in alpha but P2→P1 dominance in theta. What might this mean?